<p style="text-align: center; color: blue">TRABALHO PRÁTICO</p>

<i>INTRODUÇÃO A APRENDIZAGEM DE MAQUINA</i>

# Crimes em São Francisco

Nosso trabalho será resolver o problema proposto em Crimes em São Francisco. Mais precisamente, dados um conjunto de características sobre um crime ocorrido em São Francisco, desejamos classificá-lo corretamente.


<i>Nome: Fernanda de Siqueira Teixeira</i>

## Informações prévias:


Datas  - registro de data e hora do incidente do crime

Categoria  - categoria do incidente de crime (somente em train.csv). Essa é a variável de destino que você irá prever.

Descrição  - descrição detalhada do incidente do crime (apenas em train.csv)

DayOfWeek - o dia da semana

PdDistrict - nome do distrito do departamento de polícia

Resolução - como o incidente do crime foi resolvido (apenas no train.csv)

Endereço - o endereço aproximado do incidente do crime 

X - Longitude

Y - Latitude

<i>VAMOS A PRÁTICA...</i>

In [ ]:
from PIL import Image , ImageFilter

In [ ]:
im = Image . open ( 'etapas.jpeg' )
im

## Importando as bibliotecas a serem utilizadas

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
import matplotlib
import sys

import matplotlib.pyplot as plt
from scipy.stats import norm, skew
from scipy.stats.stats import pearsonr

## 1. Conhecimento dos dados

## Carregando dataset

In [ ]:
# Utilizando a biblioteca pandas para importação dos dados
train_sf = pd.read_csv('crime_train.csv')

In [ ]:
train_sf

In [ ]:
test_sf = pd.read_csv('crime_test.csv')

In [ ]:
test_sf

### Utilizando gráficos

**Visualizando os crimes mais comuns**

In [ ]:
train_sf.groupby("Category")["Category"].count().sort_values(ascending=False)

In [ ]:
top_crimes = train_sf.Category.value_counts()[:10]
plt.figure(figsize=(12, 8))
pos = np.arange(len(top_crimes))
plt.barh(pos, top_crimes.values, color='blue');
plt.yticks(pos, top_crimes.index);

**Visualizando os endereços mais comuns**

In [ ]:
top_addresses = train_sf.Address.value_counts()[:15]
plt.figure(figsize=(12, 8))

pos = np.arange(len(top_addresses))
plt.bar(pos, top_addresses.values)
plt.xticks(pos, top_addresses.index, rotation = 70)
plt.title('Top 15 localizações com mais crimes')
plt.xlabel('Local')
plt.ylabel('Número de crimes')
plt.show()

**Visualizando os dias mais comuns**

In [ ]:
top_days = train_sf.DayOfWeek.value_counts()
plt.figure(figsize=(12, 8))

pos = np.arange(len(top_days))
plt.bar(pos, top_days.values)
plt.xticks(pos, top_days.index, rotation = 70)
plt.title('Crimes por dia')
plt.xlabel('Dia da semana')
plt.ylabel('Número de crimes')
plt.show()

## 2. Pré-processamento:

## Melhorando o dataset para melhor entendimento: 

In [ ]:
train_data = pd.DataFrame(train_sf) 

#renomear colunas para melhor entendimento
train_data = train_data.rename(columns={'Dates': 'Datas', 'Category': 'Categoria', 'Descript': 'Descrição', 'DayOfWeek': 'Dias da Semana', 'PdDistrict': 'Distrito', 'Resolution': 'Resolução', 'Address': 'Endereço', 'X': 'Longitude', 'Y': 'Latitude'})


In [ ]:
# Verificando dados iniciais do dataset importado

train_data.head()

Observa-se que neste arquivo train_data temos a resolução e 7 features para prever a variável-alvo "Categoria"

In [ ]:
test_data = pd.DataFrame(test_sf) 

#renomear colunas para melhor entendimento

test_data = test_data.rename(columns={'Id': 'ID', 'Dates': 'Datas', 'DayOfWeek': 'Dias da Semana', 'PdDistrict': 'Distrito', 'Resolution': 'Resolução', 'Address': 'Endereço', 'X': 'Longitude', 'Y': 'Latitude'})


In [ ]:
# Verificando dados iniciais do dataset importado

test_data.head()

In [ ]:
# Verificando detalhes estatísticos do dataset

train_data.describe()

In [ ]:
# Verificando detalhes estatísticos do dataset

test_data.describe()

In [ ]:
# O atributo shape retorna uma tupla, mostrando quantas linhas e colunas temos 
train_data.shape

In [ ]:
test_data.shape

In [ ]:
# Entendendo o dataset, colunas, data types, quantidade de registros por coluna

train_data.info()

Existem ao todo 878.049 dados em nosso dataset train_data. Nenhuma coluna com dado faltante.


In [ ]:
# Entendendo o dataset, colunas, data types, quantidade de registros por coluna

test_data.info()

Existem ao todo 884.262 dados em nosso dataset test_data. Nenhuma coluna com dado faltante.

In [ ]:
# Verificar se há algum valor nulo
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
# identificar o tipo de dados
train_data.dtypes

In [ ]:
test_data.dtypes

Após análise dos dados, observa-se que o arquivo train_data é mais adequado para se trabalhar. 

O arquivo test_data não possui a variável de destino e a resolução.

## 3. Seleção das variáveis: 

## A variável alvo

In [ ]:
# Plotar um histograma

plt.hist(train_data['Categoria'])
plt.show()

A variável de destino que iremos prever é a categoria do incidente de crime, disponivel somente no arquivo train.csv.


In [ ]:
print(train_data['Categoria'])

## Transformando em target

In [ ]:
# .unique retira os valores repetidos

target = train_data["Categoria"].unique()
print(target)

Observando nossa target, vemos que é um problema de Classificação por existir vários valores categóricos. 

## Limpando os dados para começar a trabalhar

**Mapeando algumas features categoricas para features numericas**

In [ ]:
data_dict = {}
count = 1
for data in target:
    data_dict[data] = count
    count+=1
train_data["Categoria"] = train_data["Categoria"].replace(data_dict)

In [ ]:
#Substituindo os dias da semana por números:
data_week_dict = {
    "Monday": 1,
    "Tuesday":2,
    "Wednesday":3,
    "Thursday":4,
    "Friday":5,
    "Saturday":6,
    "Sunday":7
}
train_data["Dias da Semana"] = train_data["Dias da Semana"].replace(data_week_dict)
test_data["Dias da Semana"] = test_data["Dias da Semana"].replace(data_week_dict)

In [ ]:
#Substituindo cada distrito por um número:
departamento = train_data["Distrito"].unique()
data_dict_departamento = {}
count = 1
for data in departamento:
    data_dict_departamento[data] = count
    count+=1 
test_data["Distrito"] = test_data["Distrito"].replace(data_dict_departamento)
train_data["Distrito"] = train_data["Distrito"].replace(data_dict_departamento)

In [ ]:
print(train_data.head())

## Trabalhando os dados 

Vamos tentar encontrar algumas correlações entre a variável de destino e as variáveis numéricas.

Mas antes disso, temos que remover primeiro a coluna de resolução.

In [ ]:
columns_train = train_data.columns
print(columns_train)
columns_test = test_data.columns
print(columns_test)

In [ ]:
cols = columns_train.drop("Resolução")
print(cols)

In [ ]:
train_data_new = train_data[cols]
print(train_data_new.head())

In [ ]:
print(train_data_new.describe())

Todas as colunas numéricas não têm valores ausentes.

In [ ]:
corr = train_data_new.corr()
print(corr["Categoria"])

Observa-se que não há correlação forte de categoria com nenhum valor numérico.

In [ ]:
columns_train_data_new = train_data_new.columns
print(columns_train_data_new)

**Train_test_split**

Nesta fase separamos o conjunto de dados, dividindo nossa tabela em duas. Uma para utilizarmos como conjunto de treino para o nosso classificador
e um para simularmos o uso do classificador uma vez que ele tenha sido treinado.

A função train_test_split faz essa divisão para nós.



In [ ]:
from sklearn.model_selection import train_test_split


train_df = train_data_new[["Dias da Semana", "Distrito",  "Longitude", "Latitude"]]
X_treino, X_teste, y_treino, y_teste = train_test_split(train_df, train_data_new["Categoria"], random_state=0)

In [ ]:
features = ["Dias da Semana", "Distrito",  "Longitude", "Latitude"]

## 4. Escolha e aplicação do modelo de ML:

Agora iremos aplicar nossos dados ao algoritmo KNN

KNN é um método baseado em distâncias que considera a proximidade entre dados na realização de predições.

A hipótese desse algoritmo é que dados similares tendem a estar concentrados na mesma região no espaço de dispersão dos dados.

Para evitar empate entre as classes, devemos escolher um valor de "K" impar. 

## Modelando os dados

## Importando o KNN e criando nosso classificador

In [ ]:
#Importando o modelo KNN
from sklearn.neighbors import KNeighborsClassifier

# Definindo o valor de vizinhos
knn = KNeighborsClassifier(n_neighbors=5)

#Treinando o modelo, com dados de treinamento
knn.fit(X_treino, y_treino)

In [ ]:
knn.score(X_teste, y_teste)

In [ ]:
predictions = knn.predict(X_teste)

In [ ]:
predictions

In [ ]:
from collections import OrderedDict
data_dict_new = OrderedDict(sorted(data_dict.items()))
print(data_dict_new)

In [ ]:
#print(type(predictions))PARA USAR NO KAGGLE
result_dataframe = pd.DataFrame({
    "Id": test_data["ID"]
})
for key,value in data_dict_new.items():
    result_dataframe[key] = 0
count = 0
for item in predictions:
    for key,value in data_dict.items():
        if(value == item):
            result_dataframe[key][count] = 1
    count+=1
result_dataframe.to_csv("submission_knn.csv", index=False)

In [ ]:
knn.predict(X_teste.iloc[0:2])

## 5. Medição dos resultados:

## Avaliando o Algoritmo

**Analisando e validando os resultados obtidos**


In [ ]:
# Importando métricas para validação do modelo
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Imprimindo a matriz confusa
print("Matriz Confusa: ")
print(confusion_matrix(y_teste, predictions))

In [ ]:
# Imprimindo o relatório de classificação
print("Relatório de classificação: \n", classification_report(y_teste, predictions))

In [ ]:
# Imprimindo o quão acurado foi o modelo
print('Acurácia do modelo: ' , accuracy_score(y_teste, predictions))

**Loop para gerar testes com diferentes valores de vizinho (K)**


In [ ]:
error = []

# Calculating error for K values between 1 and 12
for i in range(1, 12):  
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_treino, y_treino)
    pred_i = knn.predict(X_teste)
    error.append(np.mean(pred_i != y_teste))

**Comparando o Error Rate gerado de valores K diferentes**


In [ ]:
plt.figure(figsize=(12, 6))  
plt.plot(range(1, 10), error, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('Error Rate K Value')  
plt.xlabel('K Value')  
plt.ylabel('Mean Error')  

## 6. Aperfeiçoamento

## Aplicando melhor parâmetro para K encontrado

In [ ]:
# Treinando o modelo KNN com o melhor parâmetro para K

from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=9)  
classifier.fit(X_treino, y_treino)  

In [ ]:
# Aplicando os valores de teste novamente
y_pred = classifier.predict(X_teste)

In [ ]:
# Importando métricas para validação do modelo
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Imprimindo a matriz confusa
print("Matriz Confusa: ")
print(confusion_matrix(y_teste, y_pred), "\n")

In [ ]:
# Imprimindo o relatório de classificação
print("Relatório de classificação: \n", classification_report(y_teste, y_pred))

In [ ]:
# Imprimindo o quão acurado foi o modelo
print('Acurácia do modelo: ' , accuracy_score(y_teste, y_pred))

### Árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
decision_tree = DecisionTreeClassifier(max_depth=3)

In [ ]:
decision_tree.fit(X_treino, y_treino)

In [ ]:
decision_tree.score(X_teste, y_teste)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plot_dataset(decision_tree)

# CONCLUSÃO: 

**SOBRE O ALGORITMO UTILIZADO:**

O KNN é um modelo simples e muito eficaz.

<i>A favor do KNN temos:</i>

1 – É um dos mais simples algorítimos a se implementar.

2- Boa precisão na maioria dos casos aplicados.

3 – A facilidade para se efetuar tunning dos poucos parametros existentes (Valor de K e Medida de Distância)

4 – Tempo para se efetuar treinamento dos dados também é diferenciado, sendo um dos mais rápidos para esta atividade.

5 – O KNN pode ser utilizado em dados não lineares bem como para problemas de regressão.


<i>Contra o KNN temos:</i>

1 – Uma demora excessiva na fase de teste e o alto consumo de memória para realizar esta atividade de teste, uma vez que o mesmo armazena todo dataset em memoria.

2 – KNN não é indicado a dados de grandes dimensões, imagina ter que calcular a distância de todos os pontos de dados entre si, quanto mais dados e mais dimensões desses registros, maior será o tempo para processar os cálculos.

3 – Uma vez que o mesmo trabalha com medidas de distância necessita sempre que nos atentemos a escala dos valores utilizados, para que não gere resultados equivocados.

**SOBRE O TRABALHO APRESENTADO:**

- O trabalho foi realizado com o objetivo de entender melhor a disciplina.
- O problema escolhido pela aluna trata-se de dados de um conjunto de características sobre um crime ocorrido em São Francisco de 2003 a 2015.
- O KNN se mostrou mais eficiente com k=9, com acurácia de 24%.
